In [14]:
import numpy as np
import time
import pandas as pd
import datetime as dt

In [2]:
dfe = pd.read_csv("Emp_Current_Extract_14thApril.csv",low_memory=False)
pd.set_option('display.max_columns', None)

In [3]:
dfe['FLIGHT_NUM']= dfe['marketing_carrier_id'] +  dfe['flight_number'].astype(str)
dfe['FLIGHT_NUM']= dfe['marketing_carrier_id'] +  dfe['flight_number'].astype(str)
dfe.replace('(null)',np.nan, inplace=True)
dfe['FLIGHT_NUM2']=dfe['FLIGHT_NUM']
dfe['FLIGHT_NUM2'] = dfe['FLIGHT_NUM2'].str.replace('QF', 'QFA')

C:\Users\cxcto\AppData\Local\Temp\ipykernel_1720\2868030904.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfe.replace('(null)',np.nan, inplace=True)


In [5]:
# !pip install openmeteo-requests
# !pip install requests-cache retry-requests numpy pandas

In [6]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [7]:
import os
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", "airports.dat")

('airports.dat', <http.client.HTTPMessage at 0x26d800b6d10>)

In [8]:
colnames=['id', 'airport_name', 'city', 'country','iata','icao','latitude','longitude','altitude/feet','timezone','DST','Tz_database','type','source'] 
df_air = pd.read_csv('airports.dat', delimiter=',', names=colnames, header=None)
df_air = df_air.reset_index()
df_air1 = df_air[['city','iata','latitude','longitude']]

In [9]:
dfe = dfe.merge(df_air1, left_on='departure_airport_id',right_on='iata')

In [10]:
dfe = dfe.drop(columns=['iata'])

In [11]:
dfe.rename(columns={'city': 'dep_city', 'latitude': 'dep_latitude','longitude': 'dep_longitude'}, inplace=True)

In [12]:
dfe = dfe.merge(df_air1, left_on='arrival_airport_id',right_on='iata')
dfe = dfe.drop(columns=['iata'])
dfe.rename(columns={'city': 'arr_city', 'latitude': 'arr_latitude','longitude': 'arr_longitude'}, inplace=True)

In [13]:
dfe['scheduled_gate_departure'].fillna(dfe['planned_departure'])
dfe['scheduled_gate_departure'].fillna(dfe['actual_runway_departure'],inplace=True)
dfe['scheduled_gate_departure'].isna().sum()
dfe.dropna(subset=['scheduled_gate_departure'],inplace=True)
dfe['scheduled_gate_arrival'] = pd.to_datetime(dfe['scheduled_gate_arrival'])
dfe['scheduled_gate_arrival'].fillna(dfe['estimated_gate_arrival'],inplace=True)
dfe['scheduled_gate_arrival'].fillna(dfe['actual_gate_arrival'],inplace=True)
dfe.dropna(subset=['scheduled_gate_arrival'],inplace=True)
dfe['scheduled_gate_arrival'] = pd.to_datetime(dfe['scheduled_gate_arrival'])

C:\Users\cxcto\AppData\Local\Temp\ipykernel_1720\2770148334.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfe['scheduled_gate_departure'].fillna(dfe['actual_runway_departure'],inplace=True)
C:\Users\cxcto\AppData\Local\Temp\ipykernel_1720\2770148334.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [15]:
def weather_api(lat,lon,start,end):
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
    	"latitude": lat,
    	"longitude": lon,
    	"start_date": str(start),
    	"end_date": str(end),
    	"hourly": ["temperature_2m", "precipitation", "rain", "wind_speed_10m", "wind_speed_100m"]
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
    hourly_rain = hourly.Variables(2).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(4).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
    	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = hourly.Interval()),
    	inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    
    hourly_dataframe = pd.DataFrame(data = hourly_data)
    # change the datetime format
    hourly_dataframe['date'] =hourly_dataframe['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return hourly_dataframe

In [16]:
### new column for depeature
dfe['dep_precipitation_12h'] = np.nan
dfe['dep_rain_12h'] = np.nan
dfe['dep_max_temp'] = np.nan
dfe['dep_wind_10m'] = np.nan
dfe['dep_wind_100m'] = np.nan

### new column for arrival
dfe['arr_precipitation_12h'] = np.nan
dfe['arr_rain_12h'] = np.nan
dfe['arr_max_temp'] = np.nan
dfe['arr_wind_10m'] = np.nan
dfe['arr_wind_100m'] = np.nan

In [17]:
data = []
dfc = pd.DataFrame(data, columns=['lat', 'lon','start','end'])

In [18]:
def append_row(df, row):
    return pd.concat([
                df, 
                pd.DataFrame([row], columns=row.index)]
           ).reset_index(drop=True)

In [19]:
for i in range(0,len(dfe)):

    ## depature weather
    dep_lat = dfe.iloc[i]['dep_latitude']
    dep_lon = dfe.iloc[i]['dep_longitude']
    
    dep_end = pd.to_datetime(dfe.iloc[i]['scheduled_gate_departure']).round(freq='h')
    dep_start = (dep_end- pd.Timedelta(hours=12, minutes=0, seconds=0))
    dep_end1 = pd.to_datetime(dep_end)
    dep_start1 = pd.to_datetime(dep_start)
    dep_end = dep_end1.date()
    dep_start = dep_start1.date()
    new_row = pd.Series({'lat':dep_lat, 'lon':dep_lon, 'start': dep_start,'end':dep_end})
    dfc = append_row(dfc, new_row)

    ##arr
    arr_lat = dfe.iloc[i]['arr_latitude']
    arr_lon = dfe.iloc[i]['arr_longitude']
    
    arr_end = pd.to_datetime(dfe.iloc[i]['scheduled_gate_arrival']).round(freq='h')
    arr_start = (arr_end- pd.Timedelta(hours=12, minutes=0, seconds=0))
    arr_end1 = pd.to_datetime(arr_end)
    arr_start1 = pd.to_datetime(arr_start)
    arr_end = arr_end1.date()
    arr_start = arr_start1.date()

    new_row = pd.Series({'lat':arr_lat, 'lon':arr_lon, 'start': arr_start,'end':arr_end})
    dfc = append_row(dfc, new_row)

C:\Users\cxcto\AppData\Local\Temp\ipykernel_1720\1039822126.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([


In [20]:
dfc.head()

,lat,lon,start,end
0,7.461870,151.843002,2024-04-13,2024-04-14
1,13.483400,144.796005,2024-04-13,2024-04-14
2,-27.384199,153.117004,2024-04-13,2024-04-14
3,-33.946098,151.177002,2024-04-13,2024-04-14
4,-16.885799,145.755005,2024-04-13,2024-04-14


In [21]:
tp = dfc.groupby(by=["lat","lon","start","end"]).count()
tp = tp.reset_index()
tp.head()

,lat,lon,start,end
0,-54.8433,-68.2958,2024-04-14,2024-04-14
1,-54.8433,-68.2958,2024-04-14,2024-04-15
2,-54.8433,-68.2958,2024-04-15,2024-04-15
3,-53.7777,-67.7494,2024-04-14,2024-04-14
4,-53.7777,-67.7494,2024-04-15,2024-04-15


In [23]:
dfe['scheduled_gate_departure'] = pd.to_datetime(dfe['scheduled_gate_departure'])
dfe['scheduled_gate_arrival'] = pd.to_datetime(dfe['scheduled_gate_arrival'])

In [24]:
record =0

In [28]:
# Initialize an empty list to store the rain sums
for i in range(793,9000):
    if (i == 1000) or (i== 2000) or (i ==3000) or (i == 4000) or(i == 5000) or (i == 6000) or (i == 7000) or (i == 8000) or (i == 9000):
        print(i)

    record = i
    lat = tp.iloc[i]['lat']
    lon = tp.iloc[i]['lon']
    start= tp.iloc[i]['start']
    end = tp.iloc[i]['end']
    

    end = pd.to_datetime(end)
    start = pd.to_datetime(start)
    end = end.date()
    start = start.date()

    dep_hourly = weather_api(lat,lon,start,end)
    dep_hourly['date'] = pd.to_datetime(dep_hourly['date'])
    # time.sleep(0.1)
    
    for index, row in dfe.iterrows():
        scheduled_gate_departure = row['scheduled_gate_departure']
        scheduled_gate_arrival = row['scheduled_gate_arrival']
        # scheduled_gate_departure = pd.to_datetime(scheduled_gate_departure)
        # scheduled_gate_departure = pd.to_datetime(scheduled_gate_arrival)
        dep_latitude = row['dep_latitude']
        dep_longitude = row['dep_longitude']
        arr_latitude =  row['arr_latitude']
        arr_longitude =  row['arr_longitude']
        dep_max_temp = row['dep_max_temp']
        arr_max_temp = row['arr_max_temp']
        dep_12 = scheduled_gate_departure - pd.Timedelta(hours=12)
        arr_12 = scheduled_gate_arrival - pd.Timedelta(hours=12)
       
        ################ depature updates
        if (dep_latitude== lat) & (dep_longitude == lon) & (dep_hourly['date'].iloc[-1]>= scheduled_gate_departure) & (dep_hourly['date'].iloc[0] <= dep_12):
            # print("d")
            # print(index)
           
        # Filter the first dataset for matching conditions
            matching_rows = dep_hourly[
                (dep_hourly['date'] <= scheduled_gate_departure) &
                (dep_hourly['date'] > dep_12)
            ]
            rain_sum = matching_rows['rain'].sum()    
            precipitation = matching_rows['precipitation'].sum()
            max_temperature = matching_rows['temperature_2m'].max()
            mean_wind_speed_10m = matching_rows['wind_speed_10m'].mean()
            mean_wind_speed_100m = matching_rows['wind_speed_100m'].mean()
            dfe.loc[index, 'dep_precipitation_12h'] = precipitation
            dfe.loc[index, 'dep_rain_12h'] = rain_sum
            dfe.loc[index, 'dep_max_temp'] = max_temperature
            dfe.loc[index, 'dep_wind_10m'] = mean_wind_speed_10m
            dfe.loc[index, 'dep_wind_100m'] = mean_wind_speed_100m
            # print(matching_rows)
            # print("D")
        
        ################ arrival updates
        if (arr_latitude== lat) & (arr_longitude == lon) & (dep_hourly['date'].iloc[-1]>= scheduled_gate_arrival) & (dep_hourly['date'].iloc[0] <arr_12):
            # print("a")
            # print(index)
           
        # Filter the first dataset for matching conditions
            matching_rows = dep_hourly[
    
                (dep_hourly['date'] <= scheduled_gate_arrival) &
                (dep_hourly['date'] > arr_12)
            ]
            rain_sum = matching_rows['rain'].sum()    
            precipitation = matching_rows['precipitation'].sum()
            max_temperature = matching_rows['temperature_2m'].max()
            mean_wind_speed_10m = matching_rows['wind_speed_10m'].mean()
            mean_wind_speed_100m = matching_rows['wind_speed_100m'].mean()
            dfe.loc[index, 'arr_precipitation_12h'] = precipitation
            dfe.loc[index, 'arr_rain_12h'] = rain_sum
            dfe.loc[index, 'arr_max_temp'] = max_temperature
            dfe.loc[index, 'arr_wind_10m'] = mean_wind_speed_10m
            dfe.loc[index, 'arr_wind_100m'] = mean_wind_speed_100m

1000


ConnectionError: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Max retries exceeded with url: /v1/archive?latitude=-0.1507109999656677&longitude=109.40399932861328&start_date=2024-04-13&end_date=2024-04-14&hourly=temperature_2m&hourly=precipitation&hourly=rain&hourly=wind_speed_10m&hourly=wind_speed_100m&format=flatbuffers (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026D8114DBD0>: Failed to resolve 'archive-api.open-meteo.com' ([Errno 11001] getaddrinfo failed)"))

In [26]:
dfe

,flight_id,marketing_carrier_id,operating_carrier_id,wet_lease_carrier_id,fleet_aircraft_id,service_type_iata,flight_number,tail_number,departure_airport_id,departure_utc_offset,departure_terminal,departure_gate,published_departure,published_departure_local,planned_departure,planned_departure_local,scheduled_gate_departure,scheduled_gate_departure_local,estimated_gate_departure,estimated_gate_departure_local,first_gate_departure,first_gate_departure_local,actual_gate_departure,actual_gate_departure_local,scheduled_runway_departure,scheduled_runway_departure_local,estimated_runway_departure,estimated_runway_departure_local,actual_runway_departure,actual_runway_departure_local,estimated_taxi_out_time,actual_taxi_out_time,gate_departure_delay,arrival_airport_id,arrival_utc_offset,arrival_terminal,arrival_gate,baggage_claim,published_arrival,published_arrival_local,planned_arrival,planned_arrival_local,scheduled_runway_arrival,scheduled_runway_arrival_local,estimated_runway_arrival,estimated_runway_arrival_local,actual_runway_arrival,actual_runway_arrival_local,scheduled_gate_arrival,scheduled_gate_arrival_local,estimated_gate_arrival,estimated_gate_arrival_local,actual_gate_arrival,actual_gate_arrival_local,estimated_taxi_in_time,actual_taxi_in_time,gate_arrival_delay,planned_route,departure_fix,arrival_fix,is_diverted,is_cancelled,gate_return_total_ground_time,gate_return_longest_ground_time,diverted_airport_landings,diverted_reached_destination,diverted_actual_elapsed_time,diverted_arrival_elapsed_time,diverted_distance_miles,actual_flight_duration,scheduled_block_time,actual_block_time,block_time_overshoot,delay_difference_indicator,last_processed_time,flight_date_utc,flight_date_local,FLIGHT_NUM,FLIGHT_NUM2,dep_city,dep_latitude,dep_longitude,arr_city,arr_latitude,arr_longitude,dep_precipitation_12h,dep_rain_12h,dep_max_temp,dep_wind_10m,dep_wind_100m,arr_precipitation_12h,arr_rain_12h,arr_max_temp,arr_wind_10m,arr_wind_100m
0,1246171183,UA,UA,NaN,NaN,P,3921,NaN,TKK,+10:00,NaN,NaN,NaN,NaN,2024-04-14 02:02:00,2024-04-14 02:02:00,2024-04-14 01:45:00,2024-04-14 01:45:00,2024-04-14 01:45:00,2024-04-14 01:45:00,NaN,NaN,NaN,NaN,2024-04-14 02:02:00,2024-04-14 02:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GUM,+10:00,NaN,NaN,NaN,NaN,NaN,2024-04-14 03:17:00,2024-04-14 03:17:00,2024-04-14 03:17:00,2024-04-14 03:17:00,NaN,NaN,NaN,NaN,2024-04-14 03:20:00,2024-04-14 03:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,true,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,NaN,NaN,NaN,2024-04-13 20:04:37,2024-04-13,2024-04-14,UA3921,UA3921,Chuuk,7.461870,151.843002,Agana,13.483400,144.796005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1245901050,JQ,JQ,NaN,382956,J,811,VH-OFU,BNE,+10:00,D,31,2024-04-14 07:00:00,2024-04-14 07:00:00,NaN,NaN,2024-04-14 07:00:00,2024-04-14 07:00:00,2024-04-14 07:36:00,2024-04-14 07:36:00,2024-04-14 07:03:00,2024-04-14 07:03:00,2024-04-14 07:36:00,2024-04-14 07:36:00,NaN,NaN,2024-04-14 07:47:00,2024-04-14 07:47:00,2024-04-14 07:47:00,2024-04-14 07:47:00,11,11,36,SYD,+10:00,2,A49,D5,2024-04-14 08:35:00,2024-04-14 08:35:00,NaN,NaN,NaN,NaN,2024-04-14 09:05:00,2024-04-14 09:05:00,2024-04-14 09:05:00,2024-04-14 09:05:00,2024-04-14 08:35:00,2024-04-14 08:35:00,2024-04-14 09:10:00,2024-04-14 09:10:00,2024-04-14 09:10:00,2024-04-14 09:10:00,5,5,35,NaN,NaN,NaN,false,false,17,17,NaN,NaN,NaN,NaN,NaN,78,95,94,-1,-1,2024-04-14 09:13:47,2024-04-13,2024-04-14,JQ811,JQ811,Brisbane,-27.384199,153.117004,Sydney,-33.946098,151.177002,0.0,0.0,25.317501,9.34969,12.158757,0.0,0.0,23.161001,7.319407,10.335207
2,1245919657,QF,SSQ,SSQ,65663,J,2349,VH-QOW,CNS,+10:00,2,16,2024-04-14 10:50:00,2024-04-14 10:50:00,NaN,NaN,2024-04-14 10:50:00,2024-04-14 10:50:00,2024-04-14 11:38:00,2024-04-14 11:38:00,2024-04-14 10:50:00,2024-04-14 10:50:00,2024-04-14 11:38:00,2024-04-14 11:38:00,NaN,NaN,2024-04-14 11:44:00,2024-04-14 11:44:00,2024-04-14 11:44:00,2024-04-14 11:44:00,6,6,48,TSV,+10:00,NaN,NaN,1,2024-04-14 11:45:00,2024-04-14 11:45:00,NaN,NaN,NaN,NaN,2024-04-14 12:22:0

In [29]:
print(record)

1883
